In [1]:
# Phase 2: Agentic RAG System with Multi-Agent Architecture
# Requirements: pip install gradio chromadb langchain google-generativeai openai PyPDF2 python-docx python-pptx pillow

import os
import io
import asyncio
from datetime import datetime
from dataclasses import dataclass
from enum import Enum
from typing import List, Dict, Any, Optional

import gradio as gr
import google.generativeai as genai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
import chromadb
from chromadb.config import Settings
from PIL import Image
import PyPDF2
from docx import Document
from pptx import Presentation

# ------------------------------
# Agent infrastructure
# ------------------------------
class AgentType(Enum):
    RESEARCHER = "researcher"
    SYNTHESIZER = "synthesizer"
    FACT_CHECKER = "fact_checker"
    FOLLOW_UP = "follow_up"
    COORDINATOR = "coordinator"

@dataclass
class AgentResponse:
    agent_type: AgentType
    content: str
    confidence: float
    sources: List[str]
    reasoning: str
    follow_up_needed: bool = False

@dataclass
class ResearchTask:
    query: str
    context: str
    agent_type: AgentType
    priority: int = 1
    dependencies: List[str] = None

# ------------------------------
# Agentic RAG System class
# ------------------------------
class AgenticRAG:
    def __init__(self, gemini_api_key: str, openai_api_key: str):
        """
        Initialize Agentic RAG:
        - Gemini: reasoning LLM (google.generativeai)
        - OpenAI: embeddings (OpenAIEmbeddings via langchain)
        """
        # Configure Gemini (LLM)
        genai.configure(api_key=gemini_api_key)
        self.gemini_model = genai.GenerativeModel('gemini-1.5-flash')

        # Configure OpenAI embeddings
        self.embeddings = OpenAIEmbeddings(
            model="text-embedding-3-small",
            openai_api_key=openai_api_key
        )

        # Initialize ChromaDB (persistent)
        self.chroma_client = chromadb.PersistentClient(
            path="./agentic_knowledge_base",
            settings=Settings(anonymized_telemetry=False)
        )

        # Collections
        self.text_collection = self.chroma_client.get_or_create_collection(
            name="text_documents_v2",
            metadata={"description": "Text-based documents with agent metadata"}
        )
        self.image_collection = self.chroma_client.get_or_create_collection(
            name="image_documents_v2",
            metadata={"description": "Image-based documents with agent metadata"}
        )
        self.agent_collection = self.chroma_client.get_or_create_collection(
            name="agent_responses",
            metadata={"description": "Agent responses and reasoning"}
        )

        # Text splitter for ingestion
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=800,
            chunk_overlap=150,
            length_function=len
        )

        # Session & agent memory
        self.session_memory: Dict[str, List[Dict[str, Any]]] = {}
        self.agent_memory: Dict[str, List[Dict[str, Any]]] = {}

    # ------------------------------
    # Document / image extraction
    # ------------------------------
    def extract_text_from_file(self, file_path: str, file_type: str) -> str:
        """Extract text from PDF / DOCX / PPTX / TXT"""
        try:
            if file_type == 'pdf':
                with open(file_path, 'rb') as f:
                    reader = PyPDF2.PdfReader(f)
                    pages_text = []
                    for p in reader.pages:
                        page_text = p.extract_text() or ""
                        pages_text.append(page_text)
                    return "\n".join(pages_text)

            elif file_type == 'docx':
                doc = Document(file_path)
                return "\n".join([p.text for p in doc.paragraphs])

            elif file_type == 'pptx':
                prs = Presentation(file_path)
                text = ""
                for slide in prs.slides:
                    for shape in slide.shapes:
                        if hasattr(shape, "text"):
                            text += shape.text + "\n"
                return text

            elif file_type == 'txt':
                with open(file_path, "r", encoding="utf-8") as f:
                    return f.read()

            else:
                return f"Error: Unsupported file type {file_type}"
        except Exception as e:
            return f"Error extracting text: {str(e)}"

    def process_image_with_gemini(self, image_path: str) -> str:
        """Use Gemini Vision to analyze image and return textual analysis"""
        try:
            image = Image.open(image_path)
            prompt = (
                "Analyze this image and provide:\n"
                "1. Any visible text (OCR)\n"
                "2. A detailed description of the content\n"
                "3. Key information or concepts shown\n"
                "4. Useful context for search/retrieval\n\n"
                "Format your response into clear sections."
            )

            # Gemini can accept multimodal inputs; we use generate_content with [prompt, image]
            resp = self.gemini_model.generate_content([prompt, image])
            return resp.text
        except Exception as e:
            return f"Error processing image: {str(e)}"

    # ------------------------------
    # Agents
    # ------------------------------
    async def research_agent(self, task: ResearchTask, session_id: str) -> AgentResponse:
        try:
            # Embed query
            query_vector = self.embeddings.embed_query(task.query)

            # Query text and image collections (session-scoped when available)
            where_filter = {"session_id": session_id} if session_id in self.session_memory else None

            text_results = self.text_collection.query(
                query_embeddings=[query_vector],
                n_results=5,
                where=where_filter
            )

            image_results = self.image_collection.query(
                query_embeddings=[query_vector],
                n_results=3,
                where=where_filter
            )

            all_docs = []
            sources = []

            # text_results structure: {'ids': [...], 'documents': [[...]], 'metadatas': [[...]], ...}
            if text_results.get('documents') and text_results['documents'][0]:
                for i, doc in enumerate(text_results['documents'][0]):
                    all_docs.append(doc)
                    meta = text_results.get('metadatas', [[]])[0]
                    if i < len(meta):
                        sources.append(meta[i].get('file_name', 'Unknown'))

            if image_results.get('documents') and image_results['documents'][0]:
                for i, doc in enumerate(image_results['documents'][0]):
                    all_docs.append(doc)
                    meta = image_results.get('metadatas', [[]])[0]
                    if i < len(meta):
                        sources.append(meta[i].get('file_name', 'Unknown'))

            if not all_docs:
                return AgentResponse(
                    agent_type=AgentType.RESEARCHER,
                    content="No relevant documents found in the knowledge base.",
                    confidence=0.0,
                    sources=[],
                    reasoning="Empty KB or no matches"
                )

            context = "\n\n".join(all_docs[:8])  # limit context size

            research_prompt = f"""
As a Research Agent, analyze the following documents to answer the query: "{task.query}"

Context from documents:
{context}

Your tasks:
1) Extract key information relevant to the query
2) Identify facts, figures, and important concepts
3) Note gaps in information
4) Assess reliability of the sources
5) Provide a confidence score (0-1)

Format the result clearly.
"""
            resp = self.gemini_model.generate_content(research_prompt)
            confidence = 0.8 if len(all_docs) >= 3 else 0.6 if len(all_docs) >= 1 else 0.3

            return AgentResponse(
                agent_type=AgentType.RESEARCHER,
                content=resp.text,
                confidence=confidence,
                sources=list(set(sources)),
                reasoning=f"Analyzed {len(all_docs)} documents from {len(set(sources))} sources"
            )
        except Exception as e:
            return AgentResponse(
                agent_type=AgentType.RESEARCHER,
                content=f"Research error: {str(e)}",
                confidence=0.0,
                sources=[],
                reasoning="Agent execution failed"
            )

    async def synthesis_agent(self, research_response: AgentResponse, query: str) -> AgentResponse:
        try:
            synthesis_prompt = f"""
As a Synthesis Agent, create a comprehensive answer to: "{query}"

Research findings:
{research_response.content}

Your tasks:
1) Synthesize into a coherent, well-structured answer
2) Include relevant context and connections
3) Ensure the answer directly addresses the query
"""
            resp = self.gemini_model.generate_content(synthesis_prompt)

            return AgentResponse(
                agent_type=AgentType.SYNTHESIZER,
                content=resp.text,
                confidence=min(research_response.confidence + 0.1, 1.0),
                sources=research_response.sources,
                reasoning="Synthesized information from research findings",
                follow_up_needed=len(research_response.sources) < 2
            )
        except Exception as e:
            return AgentResponse(
                agent_type=AgentType.SYNTHESIZER,
                content=f"Synthesis error: {str(e)}",
                confidence=0.0,
                sources=research_response.sources,
                reasoning="Synthesis agent execution failed"
            )

    async def fact_checker_agent(self, synthesis_response: AgentResponse, query: str) -> AgentResponse:
        try:
            fact_check_prompt = f"""
As a Fact Checker Agent, review the synthesized answer for accuracy and consistency.

Original Query: "{query}"

Synthesized Answer:
{synthesis_response.content}

Tasks:
1) Check internal consistency
2) Verify claims against the provided sources
3) Identify contradictions or uncertainties
4) Suggest corrections if needed
5) Provide a final confidence score

Return sections: VERIFIED:, UNCERTAIN:, CORRECTIONS:, CONFIDENCE:
"""
            resp = self.gemini_model.generate_content(fact_check_prompt)

            adjusted_confidence = synthesis_response.confidence
            if "UNCERTAIN" in resp.text:
                adjusted_confidence -= 0.2
            if "CORRECTIONS" in resp.text:
                adjusted_confidence -= 0.1
            adjusted_confidence = max(adjusted_confidence, 0.1)

            return AgentResponse(
                agent_type=AgentType.FACT_CHECKER,
                content=resp.text,
                confidence=adjusted_confidence,
                sources=synthesis_response.sources,
                reasoning="Fact-checked synthesized response for accuracy"
            )
        except Exception as e:
            return AgentResponse(
                agent_type=AgentType.FACT_CHECKER,
                content=synthesis_response.content,
                confidence=max(synthesis_response.confidence - 0.1, 0.0),
                sources=synthesis_response.sources,
                reasoning="Fact-checking failed, returning synthesis"
            )

    async def follow_up_agent(self, final_response: AgentResponse, query: str) -> AgentResponse:
        try:
            follow_up_prompt = f"""
As a Follow-up Agent, generate 3-5 relevant follow-up questions for the user.

Original Query: "{query}"

Answer Provided:
{final_response.content}

Make the questions specific, actionable, and aimed at exploring gaps or deeper aspects.
"""
            resp = self.gemini_model.generate_content(follow_up_prompt)
            return AgentResponse(
                agent_type=AgentType.FOLLOW_UP,
                content=resp.text,
                confidence=final_response.confidence,
                sources=final_response.sources,
                reasoning="Generated contextual follow-up questions"
            )
        except Exception as e:
            return AgentResponse(
                agent_type=AgentType.FOLLOW_UP,
                content="No follow-up questions generated due to error.",
                confidence=0.0,
                sources=[],
                reasoning="Follow-up agent execution failed"
            )

    # ------------------------------
    # Coordinator & final response formatting
    # ------------------------------
    async def coordinator_agent(self, query: str, session_id: str = "default") -> Dict[str, Any]:
        start_time = datetime.now()
        try:
            research_task = ResearchTask(query=query, context="User question requiring comprehensive analysis", agent_type=AgentType.RESEARCHER)

            research_response = await self.research_agent(research_task, session_id)
            synthesis_response = await self.synthesis_agent(research_response, query)
            fact_checked_response = await self.fact_checker_agent(synthesis_response, query)
            follow_up_response = await self.follow_up_agent(fact_checked_response, query)

            # Store agent interactions in memory
            agent_session_key = f"{session_id}_agents"
            if agent_session_key not in self.agent_memory:
                self.agent_memory[agent_session_key] = []

            interaction = {
                "timestamp": datetime.now().isoformat(),
                "query": query,
                "research": research_response.__dict__,
                "synthesis": synthesis_response.__dict__,
                "fact_check": fact_checked_response.__dict__,
                "follow_up": follow_up_response.__dict__,
                "processing_time": (datetime.now() - start_time).total_seconds()
            }
            self.agent_memory[agent_session_key].append(interaction)

            final_answer = self._create_final_response(
                fact_checked_response,
                follow_up_response,
                research_response.sources,
                interaction["processing_time"]
            )

            return {
                "answer": final_answer,
                "confidence": fact_checked_response.confidence,
                "sources": fact_checked_response.sources,
                "agent_insights": {
                    "research": research_response.reasoning,
                    "synthesis": synthesis_response.reasoning,
                    "fact_check": fact_checked_response.reasoning,
                    "follow_up": follow_up_response.reasoning
                },
                "processing_time": interaction["processing_time"],
                "follow_up_questions": follow_up_response.content
            }
        except Exception as e:
            return {
                "answer": f"Coordinator error: {str(e)}",
                "confidence": 0.0,
                "sources": [],
                "agent_insights": {"error": str(e)},
                "processing_time": (datetime.now() - start_time).total_seconds(),
                "follow_up_questions": "Unable to generate follow-up questions due to error."
            }

    def _create_final_response(self, fact_checked: AgentResponse, follow_up: AgentResponse,
                               sources: List[str], processing_time: float) -> str:
        """Format the final response for the user (cleaned, with confidence & sources)"""
        answer_lines = fact_checked.content.split('\n')
        main_answer = ""
        for line in answer_lines:
            if not any(keyword in line.upper() for keyword in ['VERIFIED:', 'UNCERTAIN:', 'CORRECTIONS:', 'CONFIDENCE:']):
                main_answer += line + "\n"

        if not main_answer.strip():
            main_answer = fact_checked.content

        stars_count = min(max(int(round(fact_checked.confidence * 5)), 0), 5)
        confidence_stars = "★" * stars_count + "☆" * (5 - stars_count)

        final_response = f"""{main_answer.strip()}

🎯 Confidence: {confidence_stars} ({fact_checked.confidence:.1%})

📚 Sources: {', '.join(set(sources)) if sources else 'No sources found'}

🤔 Follow-up Questions:
{follow_up.content}

⚡ Processing Time: {processing_time:.2f} seconds
🤖 Powered by: Multi-Agent RAG (Gemini + OpenAI embeddings)"""
        return final_response

    # ------------------------------
    # Ingestion: add document / image
    # ------------------------------
    def add_document(self, file_path: str, file_name: str, session_id: str = "default") -> str:
        """Add a document or image to the knowledge base with embeddings & metadata."""
        try:
            file_extension = file_name.lower().split('.')[-1]

            # Text documents
            if file_extension in ['pdf', 'docx', 'pptx', 'txt']:
                content = self.extract_text_from_file(file_path, file_extension)
                if not content or content.startswith("Error"):
                    return f"❌ Failed to extract content from {file_name}: {content}"

                # Split into chunks
                chunks = self.text_splitter.split_text(content)
                if not chunks:
                    return f"❌ No chunks created for {file_name}"

                # Create embeddings for all chunks in batch
                try:
                    chunk_embeddings = self.embeddings.embed_documents(chunks)
                except Exception:
                    # fallback to individual queries if embed_documents not available
                    chunk_embeddings = [self.embeddings.embed_query(chunk) for chunk in chunks]

                for i, chunk in enumerate(chunks):
                    chunk_id = f"{file_name}_{i}_{datetime.now().timestamp()}"
                    metadata = {
                        "file_name": file_name,
                        "file_type": file_extension,
                        "chunk_index": i,
                        "session_id": session_id,
                        "timestamp": datetime.now().isoformat(),
                        "content_length": len(chunk),
                        "agent_processed": True,
                        "chunk_summary": (chunk[:100] + "...") if len(chunk) > 100 else chunk
                    }

                    emb = chunk_embeddings[i] if i < len(chunk_embeddings) else self.embeddings.embed_query(chunk)
                    self.text_collection.add(
                        embeddings=[emb],
                        documents=[chunk],
                        metadatas=[metadata],
                        ids=[chunk_id]
                    )

                # update session memory
                self.session_memory.setdefault(session_id, []).append({
                    "file_name": file_name,
                    "file_type": file_extension,
                    "chunks_count": len(chunks),
                    "timestamp": datetime.now().isoformat(),
                    "agent_ready": True
                })

                return f"✅ Successfully processed {file_name} ({len(chunks)} chunks) - Agent Ready"

            # Images
            elif file_extension in ['jpg', 'jpeg', 'png', 'gif', 'bmp']:
                analysis = self.process_image_with_gemini(file_path)
                if not analysis or analysis.startswith("Error"):
                    return f"❌ Failed to process image {file_name}: {analysis}"

                # embedding for image analysis text
                try:
                    emb = self.embeddings.embed_query(analysis)
                except Exception:
                    emb = self.embeddings.embed_documents([analysis])[0]

                doc_id = f"{file_name}_{datetime.now().timestamp()}"
                metadata = {
                    "file_name": file_name,
                    "file_type": "image",
                    "session_id": session_id,
                    "timestamp": datetime.now().isoformat(),
                    "agent_processed": True,
                    "image_analysis": analysis[:200] + "..." if len(analysis) > 200 else analysis
                }

                self.image_collection.add(
                    embeddings=[emb],
                    documents=[analysis],
                    metadatas=[metadata],
                    ids=[doc_id]
                )

                self.session_memory.setdefault(session_id, []).append({
                    "file_name": file_name,
                    "file_type": "image",
                    "timestamp": datetime.now().isoformat(),
                    "agent_ready": True
                })

                return f"✅ Successfully processed image {file_name} - Agent Ready"

            else:
                return f"❌ Unsupported file type: {file_extension}"

        except Exception as e:
            return f"❌ Error processing {file_name}: {str(e)}"

    # ------------------------------
    # Session & agent info
    # ------------------------------
    def get_session_info(self, session_id: str = "default") -> str:
        if session_id not in self.session_memory:
            return "No documents uploaded in this session."

        docs = self.session_memory[session_id]
        info = f"📁 Session Documents ({len(docs)} files):\n\n"
        for doc in docs:
            info += f"• {doc['file_name']} ({doc['file_type']}) - {doc['timestamp'][:19]}\n"
            if 'chunks_count' in doc:
                info += f"  └── {doc['chunks_count']} text chunks\n"
            if doc.get('agent_ready'):
                info += f"  └── 🤖 Agent-ready processing ✓\n"

        agent_key = f"{session_id}_agents"
        if agent_key in self.agent_memory:
            interactions = len(self.agent_memory[agent_key])
            avg_time = sum(i["processing_time"] for i in self.agent_memory[agent_key]) / interactions
            info += f"\n🤖 Agent Statistics:\n• Total queries processed: {interactions}\n• Average processing time: {avg_time:.2f} seconds\n"

        return info

    def get_agent_insights(self, session_id: str = "default") -> str:
        agent_key = f"{session_id}_agents"
        if agent_key not in self.agent_memory or not self.agent_memory[agent_key]:
            return "No agent interactions recorded for this session."

        latest = self.agent_memory[agent_key][-1]
        insights = f"🔍 Latest Agent Analysis:\n\n"
        insights += f"Query: {latest['query']}\n\n"
        insights += f"🔬 Research Agent: {latest['research']['reasoning']}\n"
        insights += f"  └── Confidence: {latest['research']['confidence']:.1%}\n"
        insights += f"  └── Sources: {len(latest['research']['sources'])}\n\n"
        insights += f"🧩 Synthesis Agent: {latest['synthesis']['reasoning']}\n"
        insights += f"  └── Confidence: {latest['synthesis']['confidence']:.1%}\n\n"
        insights += f"✅ Fact Checker: {latest['fact_check']['reasoning']}\n"
        insights += f"  └── Final Confidence: {latest['fact_check']['confidence']:.1%}\n\n"
        insights += f"❓ Follow-up Agent: {latest['follow_up']['reasoning']}\n\n"
        insights += f"⚡ Total Processing Time: {latest['processing_time']:.2f} seconds\n"
        return insights

# ------------------------------
# Global instance & UI wrappers
# ------------------------------
agentic_rag_system: Optional[AgenticRAG] = None

def initialize_agentic_system(gemini_key: str, openai_key: str) -> str:
    """Initialize the Agentic RAG system with Gemini + OpenAI keys"""
    global agentic_rag_system
    if not gemini_key or not openai_key:
        return "❌ Please provide both Gemini and OpenAI API keys."
    try:
        agentic_rag_system = AgenticRAG(gemini_key.strip(), openai_key.strip())
        return "✅ Agentic RAG System initialized successfully! Multi-agent workflow ready."
    except Exception as e:
        return f"❌ Error initializing agentic system: {str(e)}"

def upload_document_agentic(files, session_id: str = "default") -> str:
    """Handle document upload from Gradio (files is a list of uploaded files)"""
    if agentic_rag_system is None:
        return "❌ Please initialize the agentic system with your API keys first."

    if not files:
        return "❌ No files uploaded."

    results = []
    for f in files:
        # Gradio file object typically provides a .name path to temp file
        file_path = getattr(f, "name", None) or f
        file_name = os.path.basename(file_path)
        try:
            result = agentic_rag_system.add_document(file_path, file_name, session_id)
        except Exception as e:
            result = f"❌ Error processing {file_name}: {str(e)}"
        results.append(result)

    return "\n".join(results)

async def ask_question_agentic(question: str, session_id: str = "default") -> str:
    if agentic_rag_system is None:
        return "❌ Please initialize the agentic system with your API keys first."
    if not question:
        return "❌ Please ask a question."

    response = await agentic_rag_system.coordinator_agent(question, session_id)
    return response["answer"]

def ask_question_sync(question: str, session_id: str = "default") -> str:
    try:
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        result = loop.run_until_complete(ask_question_agentic(question, session_id))
        loop.close()
        return result
    except Exception as e:
        return f"❌ Error processing question: {str(e)}"

def get_session_status_agentic(session_id: str = "default") -> str:
    if agentic_rag_system is None:
        return "❌ Agentic system not initialized"
    return agentic_rag_system.get_session_info(session_id)

def get_agent_insights_ui(session_id: str = "default") -> str:
    if agentic_rag_system is None:
        return "❌ Agentic system not initialized"
    return agentic_rag_system.get_agent_insights(session_id)

# ------------------------------
# Gradio UI
# ------------------------------
def create_agentic_gradio_interface():
    with gr.Blocks(title="Enterprise Knowledge Assistant - Phase 2 (Agentic)", theme=gr.themes.Soft()) as demo:
        gr.Markdown("""
# 🧠🤖 Enterprise Knowledge Assistant (Phase 2 - Agentic)

**Multi-Agent RAG System**

- Gemini → reasoning, synthesis, fact-check, follow-ups  
- OpenAI → embeddings for retrieval
        """)

        with gr.Tab("🔧 Setup"):
            gr.Markdown("### Initialize the Agentic RAG System")
            gemini_input = gr.Textbox(label="Gemini API Key", placeholder="Enter Gemini API key", type="password")
            openai_input = gr.Textbox(label="OpenAI API Key", placeholder="Enter OpenAI API key", type="password")
            init_btn = gr.Button("Initialize Agentic System", variant="primary")
            init_status = gr.Textbox(label="Status", interactive=False)
            init_btn.click(fn=initialize_agentic_system, inputs=[gemini_input, openai_input], outputs=[init_status])

        with gr.Tab("📁 Upload Documents"):
            gr.Markdown("### Upload documents/images for ingestion")
            session_input = gr.Textbox(label="Session ID", value="default", placeholder="Session ID (optional)")
            file_upload = gr.Files(label="Upload files", file_count="multiple",
                                   file_types=[".pdf", ".docx", ".pptx", ".txt", ".jpg", ".jpeg", ".png", ".gif", ".bmp"])
            upload_btn = gr.Button("Process Documents (Agentic)", variant="primary")
            upload_status = gr.Textbox(label="Upload Status", interactive=False, lines=6)
            upload_btn.click(fn=upload_document_agentic, inputs=[file_upload, session_input], outputs=[upload_status])

        with gr.Tab("🤖 Ask Questions (Multi-Agent)"):
            gr.Markdown("### Query your knowledge base")
            session_query = gr.Textbox(label="Session ID", value="default")
            question_input = gr.Textbox(label="Your Question", placeholder="Ask about uploaded documents...", lines=3)
            ask_btn = gr.Button("Get Multi-Agent Answer", variant="primary")
            answer_output = gr.Textbox(label="Agent-Generated Answer", interactive=False, lines=15)
            ask_btn.click(fn=ask_question_sync, inputs=[question_input, session_query], outputs=[answer_output])

        with gr.Tab("🔍 Agent Insights"):
            gr.Markdown("### Agent analysis & reasoning")
            insights_session_input = gr.Textbox(label="Session ID", value="default")
            insights_btn = gr.Button("Get Agent Insights", variant="secondary")
            insights_output = gr.Textbox(label="Agent Reasoning and Analysis", interactive=False, lines=12)
            insights_btn.click(fn=get_agent_insights_ui, inputs=[insights_session_input], outputs=[insights_output])

        with gr.Tab("📊 Session Info"):
            gr.Markdown("### Session status & memory")
            session_status_input = gr.Textbox(label="Session ID", value="default")
            status_btn = gr.Button("Check Status")
            status_output = gr.Textbox(label="Session Information", interactive=False, lines=10)
            status_btn.click(fn=get_session_status_agentic, inputs=[session_status_input], outputs=[status_output])

    return demo

if __name__ == "__main__":
    demo = create_agentic_gradio_interface()
    demo.launch(server_name="0.0.0.0", server_port=7862, share=True)


/Users/utsabchakraborty/Documents/Agentic_Class_Live/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://0.0.0.0:7862
* Running on public URL: https://e72407edced9e51655.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/var/folders/9d/7gw2rnq56ls5m4r1vbglz5r00000gn/T/ipykernel_42922/924450648.py:65: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  self.embeddings = OpenAIEmbeddings(
